# Lab 2: Vektorizacija i predprocesiranje podataka

## Učitavanja Penguins skupa podataka

In [1]:
import seaborn as sns
import pandas as pd
df = sns.load_dataset('penguins')
print(f'Originalni oblik: {df.shape}')
df.head()

Originalni oblik: (344, 7)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [2]:
df_encoded = pd.get_dummies(df, columns=['species','island', 'sex'], dummy_na=False)
print(f'Oblik nakon vektorizacije kategorickih kolona: {df_encoded.shape}')
df_encoded.head()

Oblik nakon vektorizacije kategorickih kolona: (344, 12)


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
0,39.1,18.7,181.0,3750.0,True,False,False,False,False,True,False,True
1,39.5,17.4,186.0,3800.0,True,False,False,False,False,True,True,False
2,40.3,18.0,195.0,3250.0,True,False,False,False,False,True,True,False
3,NaN,NaN,NaN,NaN,True,False,False,False,False,True,False,False
4,36.7,19.3,193.0,3450.0,True,False,False,False,False,True,True,False


In [3]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
num_cols = df_encoded.select_dtypes(include=['float64','int64']).columns.tolist()
print('Nedostajuće vrednosti prije imputacije:')
print(df_encoded[num_cols].isnull().sum())
df_encoded[num_cols] = imp.fit_transform(df_encoded[num_cols])
print('Nedostajuće vrednosti nakon imputacije:')
print(df_encoded[num_cols].isnull().sum())

Nedostajuće vrednosti prije imputacije:
bill_length_mm       2
bill_depth_mm        2
flipper_length_mm    2
body_mass_g          2
dtype: int64
Nedostajuće vrednosti nakon imputacije:
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64


In [8]:
import numpy as np
mean_bl = df_encoded['bill_length_mm'].mean()
std_bl = df_encoded['bill_length_mm'].std()
mask = np.abs(df_encoded['bill_length_mm'] - mean_bl) < 3 * std_bl
df_clean = df_encoded[mask].copy()
print(f'Broj redova nakon uklanjanja outliera: {df_clean.shape[0]}')

Broj redova nakon uklanjanja outliera: 344


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_clean[num_cols] = scaler.fit_transform(df_clean[num_cols])
df_clean.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_Female,sex_Male
0,-8.870812e-01,7.877425e-01,-1.422488,-0.565789,True,False,False,False,False,True,False,True
1,-8.134940e-01,1.265563e-01,-1.065352,-0.503168,True,False,False,False,False,True,True,False
2,-6.663195e-01,4.317192e-01,-0.422507,-1.192003,True,False,False,False,False,True,True,False
3,-1.307172e-15,1.806927e-15,0.000000,0.000000,True,False,False,False,False,True,False,False
4,-1.328605e+00,1.092905e+00,-0.565361,-0.941517,True,False,False,False,False,True,True,False


---

## Zadatak: priprema i obrada Adult skupa podataka

1. **Učitavanje realnog skupa**: Preuzmite Adult dataset sa UCI: https://archive.ics.uci.edu/ml/datasets/adult
2. **Vektorizacija kategorickih podataka**: Primijeniti `LabelEncoder` za ciljni atribut (`income`) i `pd.get_dummies` za ostale kategoričle kolone (npr. `education`, `marital-status`).
3. **Imputacija**: Zamenite `?` sa NaN i iskoristiti `SimpleImputer` sa strategijom `most_frequent` za kategorije i `mean` za numeričke atribute.
4. **Outlieri**: Definisati prag od 3 standardne devijacije za numeričke kolone (npr. `hours-per-week`) i ukloniti outliere.
5. **Skaliranje**: Primijeniti `StandardScaler` ili `MinMaxScaler` na numeričke atribute.
6. **Evaluacija**: Podijeliti podatke na training/test i uporediti performanse `KNeighborsClassifier` (Accuracy, Precision, Recall) prije i poslije obrade podataka (nakon koraka 2 i nakon koraka 5).

In [18]:
import pandas as pd

column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'sex',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income'
]
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
df = pd.read_csv(url, names=column_names, na_values=' ?', skipinitialspace=True)
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [20]:
from sklearn.preprocessing import LabelEncoder

X = df.drop('income', axis=1).copy()
Y = LabelEncoder().fit_transform(df['income'])

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [22]:
cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['float64', 'int64']).columns

In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

def prepare(df):
    df_copy = df.copy()
    num_imputer = SimpleImputer(strategy='mean')
    df_copy[num_cols] = num_imputer.fit_transform(df_copy[num_cols])

    for col in cat_cols:
        df_copy[col] = df_copy[col].fillna('Missing')
        df_copy[col] = LabelEncoder().fit_transform(df_copy[col])
    
    return df_copy

X_train_p = prepare(X_train)
X_test_p = prepare(X_test)


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_p, Y_train)
Y_pred = knn.predict(X_test_p)

print('--- Raw Data Performance ---')
print(f'Accuracy:  {accuracy_score(Y_test, Y_pred):.4f}')
print(f'Precision: {precision_score(Y_test, Y_pred):.4f}')
print(f'Recall:    {recall_score(Y_test, Y_pred):.4f}')

--- Raw Data Performance ---
Accuracy:  0.7781
Precision: 0.5700
Recall:    0.3265


In [32]:
df_new = df.copy()
df_new.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [37]:
num_imputer = SimpleImputer(strategy='mean')
df_new[num_cols] = num_imputer.fit_transform(df_new[num_cols])
cat_imputer = SimpleImputer(strategy='most_frequent')
df_new[cat_cols] = cat_imputer.fit_transform(df_new[cat_cols])

In [40]:
import numpy as np

hpw = df_new['hours_per_week']
mask = np.abs(hpw - hpw.mean()) < 3*+ hpw.std()
df_new = df_new[mask]

df_new = pd.get_dummies(df_new, columns=cat_cols, drop_first=True)

In [ ]:
scaler = StandardScaler()
df_new[num_cols] = scaler.fit_transform(df_new[num_cols])

In [41]:
X = df_new.drop('income', axis=1)
Y = LabelEncoder().fit_transform(df_new['income'])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_test)

print('\n--- Prepared Data Performance ---')
print(f'Accuracy:  {accuracy_score(Y_test, Y_pred):.4f}')
print(f'Precision: {precision_score(Y_test, Y_pred):.4f}')
print(f'Recall:    {recall_score(Y_test, Y_pred):.4f}')


--- Prepared Data Performance ---
Accuracy:  0.7800
Precision: 0.5888
Recall:    0.3318
